In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from pyntcloud import PyntCloud

In [ ]:
torch.set_default_device('cuda')
torch.cuda.get_device_name(0)

In [ ]:
class VoxelDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return torch.from_numpy(sample).float()

In [ ]:
# Define the autoencoder model using PyTorch
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2, padding=0),
            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2, padding=0)
        )
        self.decoder = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv3d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv3d(32, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
directory = "abc-dataset-ply/"
files = sorted(list(filename for filename in os.listdir(directory)))
dataset = []
print(files)

In [ ]:
def convert_to_binvox(path):
    point_cloud = np.loadtxt(path, skiprows=12)[:, 0:3]
    df = pd.DataFrame(data=point_cloud, columns=['x','y','z'])
    cloud = PyntCloud(df)
    voxelgrid_id = cloud.add_structure("voxelgrid", n_x=128, n_y=128, n_z=128)
    voxelgrid = cloud.structures[voxelgrid_id]
    Binary_voxel_array = voxelgrid.get_feature_vector(mode="binary")
    dataset.append(Binary_voxel_array)

In [ ]:
for i in files:
    path = os.path.join(directory, i)
    convert_to_binvox(path)

In [ ]:
train_data = dataset[:20]
test_data = dataset[20:]

In [ ]:
train_dataset = VoxelDataset(train_data)
test_dataset = VoxelDataset(test_data)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, generator=torch.Generator(device='cuda'))
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, generator=torch.Generator(device='cuda'))

In [ ]:
autoencoder = Autoencoder()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    for data in train_loader:
        optimizer.zero_grad()
        outputs = autoencoder(data.unsqueeze(1))# Add 1 channel dimension
        loss = criterion(outputs, data.unsqueeze(1))
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Test the autoencoder on the test data
autoencoder.eval()
with torch.no_grad():
    for data in test_loader:
        reconstructed = autoencoder(data.unsqueeze(1))
        # Visualize the original and reconstructed data if needed

# Save the trained model
torch.save(autoencoder.state_dict(), 'autoencoder.pth')